# Introduction

The Melbourne House Price dataset was taken from Kaggle and was posted by Tony Pino. The house data was scraped from the Domain website which is updated weekly.

# Data Exploration

Firstly, we'll load the data and have a look at the columns and try to make sense what those are

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.feature_extraction import FeatureHasher

In [2]:
melb_df_full = pd.read_csv(r'C:\Users\john.osorio\Desktop\BLACK CAT\Capstone Project\House Price Predictor\melbourne-housing-market\Melbourne_housing_FULL.csv')

In [3]:
melb_df_full.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,68 Studley St,2,h,NaN,SS,Jellis,3/09/2016,2.5,3067.0,...,1.0,1.0,126.0,NaN,NaN,Yarra City Council,-37.8014,144.9958,Northern Metropolitan,4019.0
1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra City Council,-37.7996,144.9984,Northern Metropolitan,4019.0
2,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra City Council,-37.8079,144.9934,Northern Metropolitan,4019.0
3,Abbotsford,18/659 Victoria St,3,u,NaN,VB,Rounds,4/02/2016,2.5,3067.0,...,2.0,1.0,0.0,NaN,NaN,Yarra City Council,-37.8114,145.0116,Northern Metropolitan,4019.0
4,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra City Council,-37.8093,144.9944,Northern Metropolitan,4019.0


# TODO 1: Explain the columns/features here

In [4]:
melb_df_full.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')

In [5]:
melb_df_full.describe()

,Rooms,Price,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
count,34857.000000,2.724700e+04,34856.000000,34856.000000,26640.000000,26631.000000,26129.000000,23047.000000,13742.00000,15551.000000,26881.000000,26881.000000,34854.000000
mean,3.031012,1.050173e+06,11.184929,3116.062859,3.084647,1.624798,1.728845,593.598993,160.25640,1965.289885,-37.810634,145.001851,7572.888306
std,0.969933,6.414671e+05,6.788892,109.023903,0.980690,0.724212,1.010771,3398.841946,401.26706,37.328178,0.090279,0.120169,4428.090313
min,1.000000,8.500000e+04,0.000000,3000.000000,0.000000,0.000000,0.000000,0.000000,0.00000,1196.000000,-38.190430,144.423790,83.000000
25%,2.000000,6.350000e+05,6.400000,3051.000000,2.000000,1.000000,1.000000,224.000000,102.00000,1940.000000,-37.862950,144.933500,4385.000000
50%,3.000000,8.700000e+05,10.300000,3103.000000,3.000000,2.000000,2.000000,521.000000,136.00000,1970.000000,-37.807600,145.007800,6763.000000
75%,4.000000,1.295000e+06,14.000000,3156.000000,4.000000,2.000000,2.000000,670.000000,188.00000,2000.000000,-37.754100,145.071900,10412.000000
max,16.000000,1.120000e+07,48.100000,3978.000000,30.000000,12.000000,26.000000,433014.000000,44515.00000,2106.000000,-37.390200,145.526350,21650.000000


In [6]:
melb_df_full.Rooms.count()

34857

# TODO 2: Check for missing target values and do something about it

In [7]:
melb_df_full.Price.isnull().sum()

7610

There are a sizable 7610 properties without price information. Let's just remove them and perhaps predict their price using our model later on?

In [8]:
# Remove all rows with missing price information

melb_df_full = melb_df_full[melb_df_full.Price.notnull()]

In [9]:
# Let's create another dataframe of properties with null price values (and also null observations) and test them later

melb_df_no_price = melb_df_full[melb_df_full.Price.isnull()]

# TODO 3: Set aside features with object data type first and apply some kind of imputation procedure to the missing values

Source: https://www.kaggle.com/dansbecker/handling-missing-values

In [10]:
melb_cat_features = melb_df_full.columns[melb_df_full.dtypes == 'object']
melb_cat_features

Index(['Suburb', 'Address', 'Type', 'Method', 'SellerG', 'Date', 'CouncilArea',
       'Regionname'],
      dtype='object')

In [11]:
melb_df_cat = melb_df_full.loc[:, melb_cat_features]
melb_df_cat.head()

,Suburb,Address,Type,Method,SellerG,Date,CouncilArea,Regionname
1,Abbotsford,85 Turner St,h,S,Biggin,3/12/2016,Yarra City Council,Northern Metropolitan
2,Abbotsford,25 Bloomburg St,h,S,Biggin,4/02/2016,Yarra City Council,Northern Metropolitan
4,Abbotsford,5 Charles St,h,SP,Biggin,4/03/2017,Yarra City Council,Northern Metropolitan
5,Abbotsford,40 Federation La,h,PI,Biggin,4/03/2017,Yarra City Council,Northern Metropolitan
6,Abbotsford,55a Park St,h,VB,Nelson,4/06/2016,Yarra City Council,Northern Metropolitan


For now drop the columns with object dtypes 

In [12]:
'''
melb_df_con: the 'con' stands for continuous and we will apply the imputation procedure on this df 
and combine it with the categorical df (melb_df_cat) later on
'''

melb_df_con = melb_df_full.drop(melb_cat_features, axis = 1)
melb_df_con.head()

,Rooms,Price,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
1,2,1480000.0,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,-37.7996,144.9984,4019.0
2,2,1035000.0,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,-37.8079,144.9934,4019.0
4,3,1465000.0,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,-37.8093,144.9944,4019.0
5,3,850000.0,2.5,3067.0,3.0,2.0,1.0,94.0,NaN,NaN,-37.7969,144.9969,4019.0
6,4,1600000.0,2.5,3067.0,3.0,1.0,2.0,120.0,142.0,2014.0,-37.8072,144.9941,4019.0


The remaining categorical values are Postcode, YearBuilt, Lattitude, Longitude and Propertycount. Let's check the null values in each of these features

In [13]:
melb_df_con.loc[:, ['Postcode', 'YearBuilt', 'Lattitude', 'Longtitude', 'Propertycount']].isnull().sum()

Postcode             1
YearBuilt        15163
Lattitude         6254
Longtitude        6254
Propertycount        3
dtype: int64

NOTE:
    1. I can get the postcode of that single property from Google maps
    2. YearBuilt may or may not be actually significant in the model's accuracy
    3. Lattitude and Longitude seems pretty redundant to me since we have Suburb, Address and Council Area information anyway
    4. I may be able to find out the 3 missing values from Property Count by cross referencing similar Suburb property counts

In [14]:
# Get the single property with a null postcode
melb_df_full[melb_df_full.Postcode.isnull()]

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
29483,Fawkner Lot,1/3 Brian St,3,h,616000.0,SP,Brad,6/01/2018,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# How many listed properties are in Fawkner Lot?
melb_df_full[melb_df_full.Suburb == 'Fawkner Lot']

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
29483,Fawkner Lot,1/3 Brian St,3,h,616000.0,SP,Brad,6/01/2018,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Since there is only a single entry for Fawkner Lot and also it has a lot of missing values, I'll just drop this property on the full df, the con df and the cat df for consistency

In [16]:
melb_df_full.drop(axis = 0, index = 29483, inplace = True)
melb_df_con.drop(axis = 0, index = 29483, inplace = True)
melb_df_cat.drop(axis = 0, index = 29483, inplace = True)

Lattitude and Longitude are both redundant values in our price predictor model since we have Suburb information anyway so let's just drop those two columns

In [17]:
melb_df_full.drop(['Lattitude', 'Longtitude'], axis = 1, inplace = True)
melb_df_con.drop(['Lattitude', 'Longtitude'], axis = 1, inplace = True)

Check which properties have a null Propertycount value

In [18]:
melb_df_full[melb_df_full.Propertycount.isnull()]

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Regionname,Propertycount
18523,Footscray,2/16 Stafford St,2,u,710000.0,S,Jas,15/07/2017,5.1,3011.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26888,Camberwell,2/3 Kingsley St,2,h,825000.0,VB,Jellis,11/11/2017,7.7,3124.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# Check all the properties in Footscray
melb_df_full.loc[(melb_df_full.Suburb == 'Footscray')].head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Regionname,Propertycount
4377,Footscray,4/33 Ballarat Rd,3,t,585000.0,S,Nelson,3/09/2016,6.4,3011.0,3.0,1.0,1.0,259.0,NaN,NaN,Maribyrnong City Council,Western Metropolitan,7570.0
4378,Footscray,202/51 Gordon St,1,u,85000.0,PI,Burnham,3/09/2016,6.4,3011.0,1.0,1.0,0.0,0.0,NaN,2007.0,Maribyrnong City Council,Western Metropolitan,7570.0
4379,Footscray,4 Adelaide St,4,h,1225000.0,SP,Nelson,3/12/2016,6.4,3011.0,4.0,2.0,1.0,319.0,130.0,1915.0,Maribyrnong City Council,Western Metropolitan,7570.0
4380,Footscray,11/44 Everard St,2,u,431000.0,SP,Jas,3/12/2016,6.4,3011.0,2.0,1.0,1.0,0.0,NaN,NaN,Maribyrnong City Council,Western Metropolitan,7570.0
4381,Footscray,9 Lynch St,3,h,1003000.0,S,Naison,3/12/2016,6.4,3011.0,3.0,1.0,1.0,292.0,98.0,1900.0,Maribyrnong City Council,Western Metropolitan,7570.0


In [20]:
# Check all the properties in Camberwell
melb_df_full.loc[(melb_df_full.Suburb == 'Camberwell')].head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Regionname,Propertycount
2618,Camberwell,28 Brinsley Rd,4,h,2650000.0,S,Jellis,3/09/2016,7.8,3124.0,4.0,2.0,1.0,652.0,189.0,1890.0,Boroondara City Council,Southern Metropolitan,8920.0
2619,Camberwell,2/1 Glen Iris Rd,3,t,1615000.0,S,Marshall,3/09/2016,7.8,3124.0,3.0,1.0,2.0,389.0,NaN,NaN,Boroondara City Council,Southern Metropolitan,8920.0
2621,Camberwell,1288 Toorak Rd,4,h,3225000.0,SA,RT,3/09/2016,7.8,3124.0,4.0,3.0,2.0,934.0,NaN,NaN,Boroondara City Council,Southern Metropolitan,8920.0
2623,Camberwell,1/806 Burke Rd,2,u,1017000.0,S,Jellis,3/12/2016,7.8,3124.0,2.0,1.0,2.0,0.0,NaN,1940.0,Boroondara City Council,Southern Metropolitan,8920.0
2624,Camberwell,4/2 Callanish Rd,2,u,729000.0,S,Noel,3/12/2016,7.8,3124.0,2.0,1.0,1.0,137.0,87.0,1970.0,Boroondara City Council,Southern Metropolitan,8920.0


So it's clear that the Propertycount for Footscray and Camberwell are 7570 and 8920 respectively so we'll just apply it to the missing values 

In [21]:
melb_df_full.Propertycount[18523] = 7570.0
melb_df_full.Propertycount[26888] = 8920.0
melb_df_con.Propertycount[18523] = 7570.0
melb_df_con.Propertycount[26888] = 8920.0
melb_df_cat.CouncilArea[18523] = 'Maribyrnong City Council'
melb_df_cat.Regionname[18523] = 'Western Metropolitan'
melb_df_cat.CouncilArea[26888] = 'Boroondara City Council'
melb_df_cat.Regionname[26888] = 'Southern Metropolitan'

C:\Users\john.osorio\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\john.osorio\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\john.osorio\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is s

NOTE: There are also categorical features that are float data types such as post code, year built and property count. We'll include these in the melb_df_con for now so we can apply the imputation method on them

# TODO 4: Prove that imputing the missing values will improve the performance of the model

Now we are ready to apply imputation on the missing values in the melb_df_con features but first let's see if applying imputation actually improved performance. 

Let us define a helper function that would compute the model score

In [22]:
# Calculate the model score
def compute_mae(X_train, X_test, y_train, y_test):
    model = RandomForestRegressor()
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    return mean_absolute_error(y_test, preds)

What's the score if we just simply drop columns with null values

In [23]:
# Create a List with all features within melb_df_con that has null values
features_with_null = [feature for feature in melb_df_con.columns if melb_df_con[feature].isnull().any()]
features_with_null

['Bedroom2', 'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'YearBuilt']

The list above shows features with null values so create a test dataframe with those features and compute the score

In [24]:
# Create a new data frame where columns with null values have been dropped
test_df_drop_null = melb_df_con.drop(features_with_null, axis = 1)
y = test_df_drop_null.Price
X_drop = test_df_drop_null.drop(['Price'], axis = 1)

In [25]:
# Split the data
X_drop_train, X_drop_test, y_train, y_test = train_test_split(X_drop, y, random_state = 42, test_size = 0.3)

# Score the data
print(compute_mae(X_drop_train, X_drop_test, y_train, y_test))

C:\Users\john.osorio\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


227440.8281239405


What's the new score if we apply imputation to the dataset

In [26]:
X = melb_df_con.drop(['Price'], axis = 1)

Before we apply the imputer, let's convert null values in Bedroom2 and Bathroom to ZERO

In [27]:
X.isnull().sum()

Rooms                0
Distance             0
Postcode             0
Bedroom2          6440
Bathroom          6446
Car               6823
Landsize          9264
BuildingArea     16590
YearBuilt        15162
Propertycount        0
dtype: int64

In [28]:
X.Bedroom2.fillna(0, inplace = True)
X.Bathroom.fillna(0, inplace = True)

In [29]:
X.isnull().sum()

Rooms                0
Distance             0
Postcode             0
Bedroom2             0
Bathroom             0
Car               6823
Landsize          9264
BuildingArea     16590
YearBuilt        15162
Propertycount        0
dtype: int64

In [30]:
simple_imputer = SimpleImputer()
imputed_X = simple_imputer.fit_transform(X)

In [31]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(imputed_X, y, random_state = 42, test_size = 0.3)

# Score the data
print(compute_mae(X_train, X_test, y_train, y_test))

C:\Users\john.osorio\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


194453.39905895447


Imputing the missing values significantly improves the model's performance

# TODO 5: Finalize the melb_df_con dataframe by imputing the missing values


In [32]:
melb_df_con.head()

,Rooms,Price,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Propertycount
1,2,1480000.0,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,4019.0
2,2,1035000.0,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,4019.0
4,3,1465000.0,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,4019.0
5,3,850000.0,2.5,3067.0,3.0,2.0,1.0,94.0,NaN,NaN,4019.0
6,4,1600000.0,2.5,3067.0,3.0,1.0,2.0,120.0,142.0,2014.0,4019.0


In [33]:
melb_df_con_imp = simple_imputer.fit_transform(melb_df_con)
melb_df_con_imp.shape

(27246, 11)

In [34]:
melb_df_con_imp[0]

array([2.00000000e+00, 1.48000000e+06, 2.50000000e+00, 3.06700000e+03,
       2.00000000e+00, 1.00000000e+00, 1.00000000e+00, 2.02000000e+02,
       1.56834586e+02, 1.96660915e+03, 4.01900000e+03])

In [35]:
melb_df_con_columns = melb_df_con.columns

In [36]:
melb_df_con = pd.DataFrame(melb_df_con_imp, columns = melb_df_con_columns)

In [37]:
melb_df_con.isnull().sum()

Rooms            0
Price            0
Distance         0
Postcode         0
Bedroom2         0
Bathroom         0
Car              0
Landsize         0
BuildingArea     0
YearBuilt        0
Propertycount    0
dtype: int64

No more null values, awesome! Let's set aside the melb_df_con dataframe for now. 

# TODO 6: Apply a feature hashing scheme to the categorical values to decrease the levels significantly e.g. Suburb

Source: https://towardsdatascience.com/understanding-feature-engineering-part-2-categorical-data-f54324193e63

In [38]:
# Let's see what we got
melb_df_cat.head()

,Suburb,Address,Type,Method,SellerG,Date,CouncilArea,Regionname
1,Abbotsford,85 Turner St,h,S,Biggin,3/12/2016,Yarra City Council,Northern Metropolitan
2,Abbotsford,25 Bloomburg St,h,S,Biggin,4/02/2016,Yarra City Council,Northern Metropolitan
4,Abbotsford,5 Charles St,h,SP,Biggin,4/03/2017,Yarra City Council,Northern Metropolitan
5,Abbotsford,40 Federation La,h,PI,Biggin,4/03/2017,Yarra City Council,Northern Metropolitan
6,Abbotsford,55a Park St,h,VB,Nelson,4/06/2016,Yarra City Council,Northern Metropolitan


In [39]:
# Are there significant null values?
melb_df_cat.isnull().sum()

Suburb         0
Address        0
Type           0
Method         0
SellerG        0
Date           0
CouncilArea    0
Regionname     0
dtype: int64

I think it seems prudent to just drop the Address column since in theory, all addresses must be unique for all properties which makes it similar to the index anyway

In [40]:
melb_df_cat.drop(['Address'], axis = 1, inplace = True)

Now let's see how many unique categories we have per feature

In [41]:
for n in melb_df_cat.columns:
    
    print('We have', melb_df_cat[n].unique().shape[0], 'unique', n, 'types')

We have 344 unique Suburb types
We have 3 unique Type types
We have 5 unique Method types
We have 349 unique SellerG types
We have 78 unique Date types
We have 33 unique CouncilArea types
We have 8 unique Regionname types


We'll have to apply a feature hasher to drastically reduce the number of categories

In [55]:
# For the suburb features
fh_ten = FeatureHasher(n_features=10, input_type='string')
suburb_features = fh_ten.fit_transform(melb_df_cat['Suburb'])
hashed_sub_features = suburb_features.toarray()

In [57]:
# For the seller G types
seller_features = fh_ten.fit_transform(melb_df_cat['SellerG'])
hashed_seller_features = seller_features.toarray()

# For the date sold
fh_five = FeatureHasher(n_features=5, input_type='string')
date_features = fh_five.fit_transform(melb_df_cat['Date'])
hashed_date_features = date_features.toarray()

# For the council area
fh_three = FeatureHasher(n_features=3, input_type='string')
council_features = fh_three.fit_transform(melb_df_cat['CouncilArea'])
hashed_council_features = council_features.toarray()

# For the region names
fh_two = FeatureHasher(n_features=2, input_type='string')
region_features = fh_two.fit_transform(melb_df_cat['Regionname'])
hashed_region_features = region_features.toarray()


There are three different property types so I'll just use one hot encoding on those 3